# 제 1 유형

> 데이터
* https://www.kaggle.com/mohansacharya/graduate_admissions
* 대학원 입학 가능성 예측

In [1]:
import pandas as pd

df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/admission/train.csv")

## Q1.

> Serial No. 컬럼을 제외하고 'Chance of Admit'을 종속변수, 나머지 변수를 독립변수라 할 때, 랜덤포레스트를 통해 회귀예측을 할 때 변수중요도 값을 출력하라. (시드값에 따라 순서는 달라질 수 있음)

In [2]:
df

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,246,328,110,4,4.0,2.5,9.02,1,0.81
1,100,323,113,3,4.0,4.0,8.88,1,0.79
2,79,296,95,2,3.0,2.0,7.54,1,0.44
3,53,334,116,4,4.0,3.0,8.00,1,0.78
4,444,321,114,5,4.5,4.5,9.16,1,0.87
...,...,...,...,...,...,...,...,...,...
395,47,329,114,5,4.0,5.0,9.30,1,0.86
396,14,307,109,3,4.0,3.0,8.00,1,0.62
397,28,298,98,2,1.5,2.5,7.50,1,0.44
398,149,339,116,4,4.0,3.5,9.80,1,0.96


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Serial No.         400 non-null    int64  
 1   GRE Score          400 non-null    int64  
 2   TOEFL Score        400 non-null    int64  
 3   University Rating  400 non-null    int64  
 4   SOP                400 non-null    float64
 5   LOR                400 non-null    float64
 6   CGPA               400 non-null    float64
 7   Research           400 non-null    int64  
 8   Chance of Admit    400 non-null    float64
dtypes: float64(4), int64(5)
memory usage: 28.2 KB


In [4]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor()

X = df.drop(columns=['Serial No.', "Chance of Admit"])
y = df['Chance of Admit']

model.fit(X, y)
pd.DataFrame(model.feature_importances_, index=X.columns, columns=['importance']).sort_values('importance', ascending=False)

,importance
CGPA,0.814573
GRE Score,0.085365
TOEFL Score,0.029953
SOP,0.024816
LOR,0.024122
University Rating,0.013598
Research,0.007573


# 제 1 유형

> 데이터
* https://www.kaggle.com/mohansacharya/graduate_admissions
* 대학원 입학 가능성 예측문제 ("Chance of Admit")
* 평가지표: r2 score

In [5]:
import pandas as pd

train_set = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/admission/train.csv")
test_set = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/admission/test.csv")

## 데이터 탐색

In [6]:
train_set.head()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,246,328,110,4,4.0,2.5,9.02,1,0.81
1,100,323,113,3,4.0,4.0,8.88,1,0.79
2,79,296,95,2,3.0,2.0,7.54,1,0.44
3,53,334,116,4,4.0,3.0,8.00,1,0.78
4,444,321,114,5,4.5,4.5,9.16,1,0.87


In [7]:
train_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Serial No.         400 non-null    int64  
 1   GRE Score          400 non-null    int64  
 2   TOEFL Score        400 non-null    int64  
 3   University Rating  400 non-null    int64  
 4   SOP                400 non-null    float64
 5   LOR                400 non-null    float64
 6   CGPA               400 non-null    float64
 7   Research           400 non-null    int64  
 8   Chance of Admit    400 non-null    float64
dtypes: float64(4), int64(5)
memory usage: 28.2 KB


In [8]:
test_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Serial No.         100 non-null    int64  
 1   GRE Score          100 non-null    int64  
 2   TOEFL Score        100 non-null    int64  
 3   University Rating  100 non-null    int64  
 4   SOP                100 non-null    float64
 5   LOR                100 non-null    float64
 6   CGPA               100 non-null    float64
 7   Research           100 non-null    int64  
dtypes: float64(3), int64(5)
memory usage: 6.4 KB


In [9]:
train_set.corr().style.background_gradient()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
Serial No.,1.000000,-0.145442,-0.196865,-0.097303,-0.194476,-0.003530,-0.115314,-0.008684,-0.028852
GRE Score,-0.145442,1.000000,0.839911,0.640143,0.624516,0.543964,0.833683,0.576186,0.829859
TOEFL Score,-0.196865,0.839911,1.000000,0.641866,0.626302,0.540135,0.806297,0.497987,0.793911
University Rating,-0.097303,0.640143,0.641866,1.000000,0.724695,0.601514,0.701008,0.446468,0.693555
SOP,-0.194476,0.624516,0.626302,0.724695,1.000000,0.657518,0.705512,0.414187,0.676011
LOR,-0.003530,0.543964,0.540135,0.601514,0.657518,1.000000,0.637177,0.381986,0.643639
CGPA,-0.115314,0.833683,0.806297,0.701008,0.705512,0.637177,1.000000,0.505833,0.883597
Research,-0.008684,0.576186,0.497987,0.446468,0.414187,0.381986,0.505833,1.000000,0.549273
Chance of Admit,-0.028852,0.829859,0.793911,0.693555,0.676011,0.643639,0.883597,0.549273,1.000000


## 전처리 함수 작성

In [10]:
from sklearn.preprocessing import StandardScaler

def my_preprocess(org_df, target='train', scaler=None):
    df = org_df.copy()
    df.drop(columns=['Serial No.'], inplace=True)
    if target == "train":
        sc = StandardScaler()
        df = sc.fit_transform(df)
        return df, sc
    elif target == "test":
        sc = scaler
        df = sc.transform(df)
        return df

## 독립 / 종속 변수 분리

In [11]:
X = train_set.drop(columns=['Chance of Admit'])
y = train_set['Chance of Admit']

## 분석 (kfold, 베이지안 최적화 이용)

In [12]:
!pip install bayesian-optimization

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [13]:
# K Fold
from sklearn.model_selection import KFold

# models
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.svm import SVR

# metrics
from sklearn.metrics import r2_score, mean_squared_error

# bayesian optimization
from bayes_opt import BayesianOptimization
from functools import partial

import numpy as np

### 회귀분석

In [14]:
kf = KFold(n_splits=5, shuffle=True, random_state=11)
r2_scores_lr = []
mse_lr = []
models_lr = []
for train_idx, test_idx in kf.split(X, y):
    X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
    X_valid, y_valid = X.iloc[test_idx], y.iloc[test_idx]

    X_train_proc, sc = my_preprocess(X_train)
    X_valid_proc = my_preprocess(X_valid, target="test",  scaler=sc)

    model = LinearRegression()
    model.fit(X_train_proc, y_train)

    models_lr.append(model)
    pred = model.predict(X_valid_proc)
    r2_scores_lr.append(r2_score(y_valid, pred))
    mse_lr.append(mean_squared_error(y_valid, pred))

In [15]:
np.mean(r2_scores_lr)

0.8191567040433199

In [16]:
np.mean(mse_lr)

0.0036263133399394984

### Lasso

In [47]:
def lasso_cv(alpha, max_iter, random_state, x_data=None, y_data=None, n_splits=5, output="score"):
    kf = KFold(n_splits=int(n_splits), shuffle=True, random_state=int(random_state))
    models = []
    r2_scores = []
    mses = []
    scalers = []
    for train_idx, valid_idx in kf.split(x_data, y_data):
        x_train, y_train = x_data.iloc[train_idx], y_data[train_idx]
        x_valid, y_valid = x_data.iloc[valid_idx], y_data[valid_idx]
        
        x_train, sc = my_preprocess(x_train)
        x_valid = my_preprocess(x_valid, target="test", scaler=sc)
        scalers.append(sc)

        model = Lasso(
            alpha=alpha,
            max_iter=int(max_iter),
            random_state=int(random_state)
        )
        model.fit(x_train, y_train)
        models.append(model)
        
        pred = model.predict(x_valid)
        true = y_valid
        r2_scores.append(r2_score(true, pred))
        mses.append(mean_squared_error(true, pred))
    
    if output == "score":
        return np.mean(r2_scores)
    if output == "all":
        return models, r2_scores, mses, scalers

*  튜닝과 관련없는 변수 고정

In [48]:
lasso_fixed = partial(lasso_cv, random_state=11, x_data=X, y_data=y, n_splits=5, output='score')

* 베이지안 최적화 범위 설정

In [49]:
lasso_BO = BayesianOptimization(
    lasso_fixed,
    {
        'alpha': (0.0001, 1),
        'max_iter': (1000, 100000)
    },
    random_state=11
)
lasso_BO.maximize(init_points=5, n_iter=50)

|   iter    |  target   |   alpha   | max_iter  |
-------------------------------------------------
| 1         | -0.01128  | 0.1804    | 2.928e+03 |
| 2         | -0.01128  | 0.4633    | 7.277e+04 |
| 3         | -0.01128  | 0.4203    | 4.906e+04 |
| 4         | 0.8085    | 0.01288   | 4.925e+04 |
| 5         | -0.01128  | 0.9418    | 8.523e+04 |
| 6         | -0.01128  | 0.8697    | 4.925e+04 |
| 7         | -0.01128  | 0.1371    | 6.562e+04 |
| 8         | -0.01128  | 0.6147    | 4.665e+04 |
| 9         | -0.01128  | 0.8606    | 3.835e+04 |
| 10        | 0.5609    | 0.06718   | 6.219e+04 |
| 11        | -0.01128  | 0.4027    | 2.47e+04  |
| 12        | -0.01128  | 0.9967    | 5.647e+04 |
| 13        | 0.5107    | 0.07371   | 5.152e+04 |
| 14        | -0.01128  | 0.7998    | 8.517e+04 |
| 15        | 0.2266    | 0.105     | 8.179e+03 |
| 16        | -0.01128  | 0.7748    | 8.114e+04 |
| 17        | -0.01128  | 0.4504    | 8.832e+04 |
| 18        | 0.6238    | 0.05797   | 6.219e+04 |


* 모델 학습 및 점수

In [50]:
lasso_params = lasso_BO.max["params"]
models_lasso, r2_score_lasso, mse_lasso, sc_lasso = lasso_cv(
    lasso_params["alpha"],
    lasso_params["max_iter"],
    random_state=11, x_data=X, y_data=y, n_splits=5, output="all"
)

In [21]:
np.mean(r2_score_lasso)

0.8084910767051117

In [22]:
np.mean(mse_lasso)

0.0038452610542754353

### Ridge

In [51]:
def ridge_cv(alpha, max_iter, random_state, x_data=None, y_data=None, n_splits=5, output="score"):
    kf = KFold(n_splits=int(n_splits), shuffle=True, random_state=int(random_state))
    models = []
    r2_scores = []
    mses = []
    scalers = []
    for train_idx, valid_idx in kf.split(x_data, y_data):
        x_train, y_train = x_data.iloc[train_idx], y_data[train_idx]
        x_valid, y_valid = x_data.iloc[valid_idx], y_data[valid_idx]
        
        x_train, sc = my_preprocess(x_train)
        x_valid = my_preprocess(x_valid, target="test", scaler=sc)
        scalers.append(sc)

        model = Ridge(
            alpha=alpha,
            max_iter=int(max_iter),
            random_state=int(random_state)
        )
        model.fit(x_train, y_train)
        models.append(model)
        
        pred = model.predict(x_valid)
        true = y_valid
        r2_scores.append(r2_score(true, pred))
        mses.append(mean_squared_error(true, pred))
    
    if output == "score":
        return np.mean(r2_scores)
    if output == "all":
        return models, r2_scores, mses, scalers

In [52]:
ridge_fixed = partial(ridge_cv, random_state=11, x_data=X, y_data=y, n_splits=5, output='score')

In [53]:
ridge_BO = BayesianOptimization(
    ridge_fixed,
    {
        'alpha': (0.00001, 1),
        'max_iter': (100, 1000000)
    },
    random_state=11
)
ridge_BO.maximize(init_points=5, n_iter=50)

|   iter    |  target   |   alpha   | max_iter  |
-------------------------------------------------
| 1         | 0.8192    | 0.1803    | 1.957e+04 |
| 2         | 0.8192    | 0.4632    | 7.25e+05  |
| 3         | 0.8192    | 0.4202    | 4.855e+05 |
| 4         | 0.8192    | 0.01279   | 4.874e+05 |
| 5         | 0.8192    | 0.9418    | 8.508e+05 |
| 6         | 0.8192    | 0.06033   | 3.618e+04 |
| 7         | 0.8192    | 0.75      | 8.508e+05 |
| 8         | 0.8192    | 0.8547    | 8.508e+05 |
| 9         | 0.8192    | 0.5865    | 7.251e+05 |
| 10        | 0.8192    | 0.007303  | 7.249e+05 |
| 11        | 0.8192    | 0.04133   | 4.854e+05 |
| 12        | 0.8192    | 0.5753    | 7.25e+05  |
| 13        | 0.8192    | 0.3347    | 8.507e+05 |
| 14        | 0.8192    | 0.1755    | 4.856e+05 |
| 15        | 0.8192    | 0.2404    | 7.251e+05 |
| 16        | 0.8192    | 0.7747    | 8.095e+05 |
| 17        | 0.8192    | 0.4522    | 8.094e+05 |
| 18        | 0.8192    | 0.7998    | 8.095e+05 |


In [54]:
ridge_params = ridge_BO.max["params"]
models_ridge, r2_score_ridge, mse_ridge, sc_ridge = ridge_cv(
    ridge_params["alpha"],
    ridge_params["max_iter"],
    random_state=11, x_data=X, y_data=y, n_splits=5, output="all"
)

In [55]:
np.mean(r2_score_ridge)

0.8191721112383588

In [56]:
np.mean(mse_ridge)

0.003626063185493815

### 랜덤 포레스트

In [57]:
def rf_cv(n_estimators, max_depth, n_jobs, random_state, x_data=None, y_data=None, n_splits=5, output='score'):
    kf = KFold(n_splits=int(n_splits), shuffle=True, random_state=int(random_state))
    r2_scores = []
    mses = []
    models = []
    scalers = []
    for train_idx, valid_idx in kf.split(x_data):
        x_train, y_train = x_data.iloc[train_idx], y_data[train_idx]
        x_valid, y_valid = x_data.iloc[valid_idx], y_data[valid_idx]

        x_train, sc = my_preprocess(x_train)
        x_valid = my_preprocess(x_valid, target="test", scaler=sc)
        scalers.append(sc)

        model = RandomForestRegressor(
            n_estimators=int(n_estimators),
            max_depth=int(max_depth),
            n_jobs=int(n_jobs),
            random_state=int(random_state)
        )
        model.fit(x_train, y_train)
        models.append(model)
        pred = model.predict(x_valid)
        true = y_valid
        r2_scores.append(r2_score(true, pred))
        mses.append(mean_squared_error(true, pred))

        if output == "score":
            return np.mean(r2_scores)
        if output == "all":
            return models, r2_scores, mses, scalers

In [58]:
rf_fixed = partial(rf_cv, n_jobs=-1, random_state=11, x_data=X, y_data=y, n_splits=5, output="score")

In [59]:
rf_BO = BayesianOptimization(
    rf_fixed,
    {
        'n_estimators': (100, 500),
        'max_depth': (2, 50),
    },
    random_state=11
)
rf_BO.maximize(init_points=5, n_iter=50)

|   iter    |  target   | max_depth | n_esti... |
-------------------------------------------------
| 1         | 0.7687    | 10.65     | 107.8     |
| 2         | 0.7721    | 24.23     | 390.0     |
| 3         | 0.771     | 22.17     | 294.2     |
| 4         | 0.7452    | 2.613     | 294.9     |
| 5         | 0.7724    | 47.21     | 440.3     |
| 6         | 0.7726    | 46.79     | 441.9     |
| 7         | 0.771     | 39.82     | 294.0     |
| 8         | 0.7724    | 25.98     | 439.0     |
| 9         | 0.7727    | 29.11     | 415.2     |
| 10        | 0.7763    | 7.821     | 410.5     |
| 11        | 0.7661    | 3.12      | 429.3     |
| 12        | 0.7448    | 2.671     | 393.8     |
| 13        | 0.7728    | 16.99     | 416.6     |
| 14        | 0.7723    | 34.42     | 400.8     |
| 15        | 0.772     | 39.05     | 383.6     |
| 16        | 0.7724    | 31.87     | 454.4     |
| 17        | 0.773     | 47.17     | 417.2     |
| 18        | 0.7726    | 49.53     | 462.8     |


In [61]:
rf_params = rf_BO.max["params"]
model_rf, r2_score_rf, mse_rf, sc_rf= rf_cv(
    n_estimators=rf_params["n_estimators"],
    max_depth=rf_params["max_depth"],
    n_jobs=-1, random_state=11, x_data=X, y_data=y, n_splits=5, output="all"
)

In [62]:
np.mean(r2_score_rf)

0.7784483061343925

In [63]:
np.mean(mse_rf)

0.004620446728586377

### AdaBoost

In [64]:
def ab_cv(n_estimators, learning_rate, random_state, x_data=None, y_data=None, n_splits=5, output='score'):
    kf = KFold(n_splits=int(n_splits), shuffle=True, random_state=int(random_state))
    r2_scores = []
    mses = []
    models = []
    scalers = []
    for train_idx, valid_idx in kf.split(x_data):
        x_train, y_train = x_data.iloc[train_idx], y_data[train_idx]
        x_valid, y_valid = x_data.iloc[valid_idx], y_data[valid_idx]

        x_train, sc = my_preprocess(x_train)
        x_valid = my_preprocess(x_valid, target="test", scaler=sc)
        scalers.append(sc)

        model = AdaBoostRegressor(
            n_estimators=int(n_estimators),
            learning_rate=learning_rate,
            random_state=int(random_state)
        )
        model.fit(x_train, y_train)
        models.append(model)
        pred = model.predict(x_valid)
        true = y_valid
        r2_scores.append(r2_score(true, pred))
        mses.append(mean_squared_error(true, pred))

        if output == "score":
            return np.mean(r2_scores)
        if output == "all":
            return models, r2_scores, mses, scalers

ab_fixed = partial(ab_cv, random_state=11, x_data=X, y_data=y, n_splits=5, output="score")

ab_BO = BayesianOptimization(
    ab_fixed,
    {
        'n_estimators': (1, 500),
        'learning_rate': (0.001, 1.0)
    },
    random_state=11
)
ab_BO.maximize(init_points=5, n_iter=50)

|   iter    |  target   | learni... | n_esti... |
-------------------------------------------------
| 1         | 0.745     | 0.1811    | 10.72     |
| 2         | 0.7478    | 0.4638    | 362.7     |
| 3         | 0.737     | 0.4208    | 243.2     |
| 4         | 0.7577    | 0.01377   | 244.2     |
| 5         | 0.7015    | 0.9419    | 425.5     |
| 6         | 0.7701    | 0.06126   | 19.01     |
| 7         | 0.7403    | 0.1378    | 326.7     |
| 8         | 0.7334    | 0.615     | 231.1     |
| 9         | 0.7438    | 0.01539   | 18.91     |
| 10        | 0.7507    | 0.06802   | 309.4     |
| 11        | 0.7619    | 0.02545   | 18.99     |
| 12        | 0.7395    | 0.9967    | 280.6     |
| 13        | 0.7553    | 0.08744   | 19.06     |
| 14        | 0.7501    | 0.1491    | 244.3     |
| 15        | 0.7483    | 0.1286    | 18.91     |
| 16        | 0.7403    | 0.2071    | 19.02     |
| 17        | 0.7539    | 0.03676   | 244.0     |
| 18        | 0.7536    | 0.1435    | 244.0     |


In [65]:
ab_params = ab_BO.max["params"]
model_ab, r2_score_ab, mse_ab, sc_ab = ab_cv(
    n_estimators=ab_params["n_estimators"],
    learning_rate=ab_params["learning_rate"],
    random_state=11, x_data=X, y_data=y, n_splits=5, output="all"
)
print(np.mean(r2_score_ab))
print(np.mean(mse_ab))

0.7701136721866033
0.004794264998652898


### XGBoost

In [66]:
def xgb_cv(n_estimators, max_depth, learning_rate, n_jobs, reg_alpha, reg_lambda, random_state, x_data=None, y_data=None, n_splits=5, output='score'):
    kf = KFold(n_splits=int(n_splits), shuffle=True, random_state=int(random_state))
    r2_scores = []
    mses = []
    models = []
    scalers = []
    for train_idx, valid_idx in kf.split(x_data):
        x_train, y_train = x_data.iloc[train_idx], y_data[train_idx]
        x_valid, y_valid = x_data.iloc[valid_idx], y_data[valid_idx]

        x_train, sc = my_preprocess(x_train)
        x_valid = my_preprocess(x_valid, target="test", scaler=sc)
        scalers.append(sc)

        model = XGBRegressor(
            n_estimators=int(n_estimators),
            max_depth=int(max_depth),
            learning_rate=learning_rate,
            n_jobs=int(n_jobs),
            reg_alpha=reg_alpha,
            reg_lambda=reg_lambda
        )
        model.fit(x_train, y_train)
        models.append(model)
        pred = model.predict(x_valid)
        true = y_valid
        r2_scores.append(r2_score(true, pred))
        mses.append(mean_squared_error(true, pred))

        if output == "score":
            return np.mean(r2_scores)
        if output == "all":
            return models, r2_scores, mses, scalers

xgb_fixed = partial(xgb_cv, n_jobs=-1, random_state=11, x_data=X, y_data=y, n_splits=5, output="score")

xgb_BO = BayesianOptimization(
    xgb_fixed,
    {
        'n_estimators': (1, 1024),
        'max_depth': (2, 30),
        'learning_rate': (0.0001, 0.5),
        'reg_alpha': (0, 30),
        'reg_lambda': (0, 30)
    },
    random_state=11
)
xgb_BO.maximize(init_points=5, n_iter=100)

|   iter    |  target   | learni... | max_depth | n_esti... | reg_alpha | reg_la... |
-------------------------------------------------------------------------------------
| 1         | 0.2256    | 0.09022   | 2.545     | 474.9     | 21.75     | 12.61     |
| 2         | -0.02083  | 0.2428    | 2.358     | 499.6     | 28.25     | 25.52     |
| 3         | 0.07552   | 0.365     | 5.045     | 915.5     | 25.71     | 4.953     |
| 4         | 0.6145    | 0.3162    | 2.574     | 120.4     | 9.491     | 4.737     |
| 5         | 0.6012    | 0.3795    | 24.91     | 353.6     | 9.564     | 3.35      |
| 6         | 0.5876    | 0.03651   | 23.79     | 354.0     | 10.26     | 1.697     |
| 7         | 0.7076    | 0.5       | 30.0      | 306.0     | 0.0       | 30.0      |
| 8         | 0.6914    | 0.3257    | 25.08     | 212.0     | 5.212     | 19.24     |
| 9         | -2.363    | 0.0001    | 30.0      | 20.89     | 30.0      | 30.0      |
| 10        | -1.948    | 0.0001    | 30.0      | 730.

In [67]:
xgb_params = xgb_BO.max["params"]
model_xgb, r2_score_xgb, mse_xgb, sc_xgb = xgb_cv(
    n_estimators=xgb_params["n_estimators"],
    max_depth=xgb_params["max_depth"],
    learning_rate=xgb_params["learning_rate"],
    n_jobs=-1,
    reg_alpha=xgb_params["reg_alpha"],
    reg_lambda=xgb_params["reg_lambda"],
    random_state=11, x_data=X, y_data=y, n_splits=5, output="all"
)
print(np.mean(r2_score_xgb))
print(np.mean(mse_xgb))

0.788326102535854
0.004414445901996173


### LightGBM

In [68]:
def lgbm_cv(num_leaves, learning_rate, n_estimators, subsample, colsample_bytree, reg_alpha, reg_lambda,
            random_state, x_data=None, y_data=None, n_splits=5, output='score'):
    kf = KFold(n_splits=int(n_splits), shuffle=True, random_state=int(random_state))
    r2_scores = []
    mses = []
    models = []
    scalers = []
    for train_idx, valid_idx in kf.split(x_data):
        x_train, y_train = x_data.iloc[train_idx], y_data[train_idx]
        x_valid, y_valid = x_data.iloc[valid_idx], y_data[valid_idx]

        x_train, sc = my_preprocess(x_train)
        x_valid = my_preprocess(x_valid, target="test", scaler=sc)
        scalers.append(sc)

        model = LGBMRegressor(
            num_leaves = int(num_leaves), 
            learning_rate = learning_rate, 
            n_estimators = int(n_estimators), 
            subsample = np.clip(subsample, 0, 1), 
            colsample_bytree = np.clip(colsample_bytree, 0, 1), 
            reg_alpha = reg_alpha, 
            reg_lambda = reg_lambda,
        )
        model.fit(x_train, y_train)
        models.append(model)
        pred = model.predict(x_valid)
        true = y_valid
        r2_scores.append(r2_score(true, pred))
        mses.append(mean_squared_error(true, pred))

        if output == "score":
            return np.mean(r2_scores)
        if output == "all":
            return models, r2_scores, mses, scalers

lgbm_fixed = partial(lgbm_cv, random_state=11, x_data=X, y_data=y, n_splits=5, output="score")

lgbm_BO = BayesianOptimization(
    lgbm_fixed,
    {
        'num_leaves': (16, 1024),
        'learning_rate': (0.0001, 0.1),
        'n_estimators': (16, 1024),
        'subsample': (0, 1),
        'colsample_bytree': (0, 1),
        'reg_alpha': (0, 10),
        'reg_lambda': (0, 50)
    },
    random_state=11
)
lgbm_BO.maximize(init_points=5, n_iter=100)

|   iter    |  target   | colsam... | learni... | n_esti... | num_le... | reg_alpha | reg_la... | subsample |
-------------------------------------------------------------------------------------------------------------
| 1         | 0.3225    | 0.1803    | 0.002046  | 482.9     | 746.7     | 4.202     | 24.27     | 0.01278   |
| 2         | 0.7756    | 0.4874    | 0.09419   | 873.6     | 751.8     | 1.087     | 44.7      | 0.8572    |
| 3         | 0.5841    | 0.1651    | 0.06327   | 36.65     | 133.7     | 3.164     | 7.896     | 0.759     |
| 4         | 0.78      | 0.8183    | 0.03453   | 337.3     | 128.6     | 0.8395    | 35.64     | 0.5995    |
| 5         | 0.5208    | 0.05567   | 0.04803   | 420.9     | 870.8     | 7.178     | 30.1      | 0.5524    |
| 6         | 0.3452    | 0.966     | 0.004076  | 464.6     | 94.26     | 8.634     | 42.8      | 0.07291   |
| 7         | 0.5313    | 0.942     | 0.04534   | 29.94     | 130.0     | 3.388     | 1.363     | 0.4192    |
| 8       

In [69]:
lgbm_params = lgbm_BO.max["params"]
model_lgbm, r2_score_lgbm, mse_lgbm, sc_lgbm = lgbm_cv(
    num_leaves=lgbm_params['num_leaves'], 
    learning_rate=lgbm_params['learning_rate'], 
    n_estimators=lgbm_params['n_estimators'], 
    subsample=lgbm_params['subsample'], 
    colsample_bytree=lgbm_params['colsample_bytree'], 
    reg_alpha=lgbm_params['reg_alpha'], 
    reg_lambda=lgbm_params['reg_lambda'], 
    random_state=11, x_data=X, y_data=y, n_splits=5, output="all"
)
print(np.mean(r2_score_lgbm))
print(np.mean(mse_lgbm))

0.7962738615475584
0.004248695884542016


### SVR

In [70]:
def svr_cv(kernel, degree, gamma, C, 
            random_state, x_data=None, y_data=None, n_splits=5, output='score'):
    kf = KFold(n_splits=int(n_splits), shuffle=True, random_state=int(random_state))
    r2_scores = []
    mses = []
    models = []
    scalers = []
    for train_idx, valid_idx in kf.split(x_data):
        x_train, y_train = x_data.iloc[train_idx], y_data[train_idx]
        x_valid, y_valid = x_data.iloc[valid_idx], y_data[valid_idx]

        x_train, sc = my_preprocess(x_train)
        x_valid = my_preprocess(x_valid, target="test", scaler=sc)
        scalers.append(sc)

        model = SVR(
            kernel=kernel,
            degree=int(degree),
            gamma=gamma,
            C=C
        )
        model.fit(x_train, y_train)
        models.append(model)
        pred = model.predict(x_valid)
        true = y_valid
        r2_scores.append(r2_score(true, pred))
        mses.append(mean_squared_error(true, pred))

        if output == "score":
            return np.mean(r2_scores)
        if output == "all":
            return models, r2_scores, mses, scalers

svr_fixed = partial(svr_cv, kernel='linear', random_state=11, x_data=X, y_data=y, n_splits=5, output="score")

svr_BO = BayesianOptimization(
    svr_fixed,
    {
        "degree": (1, 10),
        "gamma": (0, 50),
        "C": (0.001, 30)
    },
    random_state=11
)
svr_BO.maximize(init_points=5, n_iter=100)

|   iter    |  target   |     C     |  degree   |   gamma   |
-------------------------------------------------------------
| 1         | 0.7467    | 5.409     | 1.175     | 23.16     |
| 2         | 0.7481    | 21.75     | 4.782     | 24.27     |
| 3         | 0.748     | 0.3844    | 5.386     | 47.09     |
| 4         | 0.7467    | 25.52     | 7.57      | 5.437     |
| 5         | 0.7481    | 26.82     | 8.714     | 8.254     |
| 6         | 0.748     | 20.06     | 8.045     | 43.92     |
| 7         | 0.7466    | 14.52     | 9.963     | 17.78     |
| 8         | 0.748     | 6.205     | 7.977     | 48.81     |
| 9         | 0.7466    | 0.198     | 5.812     | 47.06     |
| 10        | 0.7478    | 20.64     | 3.706     | 36.87     |
| 11        | 0.7464    | 17.27     | 6.999     | 38.54     |
| 12        | 0.7483    | 8.607     | 5.785     | 33.72     |
| 13        | 0.7478    | 19.1      | 5.16      | 30.95     |
| 14        | 0.7482    | 4.974     | 3.462     | 12.26     |
| 15    

In [71]:
svr_params = svr_BO.max["params"]
model_svr, r2_score_svr, mse_svr, sc_svr = svr_cv(
    kernel='linear',
    degree=svr_params['degree'],
    gamma=svr_params['gamma'],
    C=svr_params['C'],
    random_state=11, x_data=X, y_data=y, n_splits=5, output='all'
)
print(np.mean(r2_score_svr))
print(np.mean(mse_svr))

0.7483043065464903
0.005249097955992104


## Ridge 성능이 가장 좋음

In [73]:
preds = []

for i, model in enumerate(models_ridge):
    test_proc = my_preprocess(test_set, target="test", scaler=sc_ridge[i])
    pred = model.predict(test_proc)
    preds.append(pred)

pred = np.mean(preds, axis=0)

pd.DataFrame(pred).to_csv("submission_pred.csv", index=False)